# **PySpark Core Program Techniques**

In [1]:
#Imports
from pyspark.sql.session import SparkSession

## **Q1. How to create SparkSession object creation & sparkContext reference?**

In [2]:
#SparkSession is the entry point for accessing the mypyspark cluster operation.
#SparkSession object will instantiate SparkContex, SqlContext, HiveContent Objects.
#|__SparkSession is a Class
#      |__builder is a Class variable/object/attribute to call the SparkSession class methods like master(), appName(), enableHiveSupport(), getOrCreate()
#           |__master('yarn')           => method to help us submit the mypyspark application to the respective cluster manager
#           |__appName('program-name')  => mypyspark program name that help us identify the jobs runs in a cluster   
#           |__enableHiveSupport()      => HiveQueryLanguage(HQL) method help us to create Catalog, UDFs, etc.,
#           |__getOrCreate()            => method to create a new SparkSession object or referring to existing SparkSession

#SparkSession Object Creation
spark=SparkSession.builder.getOrCreate()
print(spark)

25/08/06 18:41:34 WARN Utils: Your hostname, localhost.localdomain resolves to a loopback address: 127.0.0.1; using 192.168.189.130 instead (on interface ens160)
25/08/06 18:41:34 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/06 18:41:34 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
print("hellow")

hellow
